https://github.com/openai/swarm  

In [1]:
import os
import sys
import warnings; warnings.filterwarnings("ignore")
import json
import requests
import torch as th

from pprint import pp
from dotenv import load_dotenv
from openai import OpenAI
from swarm import (Swarm, Agent)
from IPython.display import (Markdown, display)

In [2]:
if sys.platform == "darwin":
    device = th.device("mps")
else:
    device = th.device("cuda" if th.cuda.is_available() else "cpu")
    
devive_cnt = th.cuda.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(f"torch = {th.__version__}")
print(f"cuda = {th.version.cuda}")

device = cuda; devive_cnt = 1
torch = 2.5.1+cu121
cuda = 12.1


In [3]:
path_project = "C:/my_project/MyGit/Machine-Learning-Column/hugging_face"
path_data = os.path.join(os.path.dirname(path_project), "data")
path_output = os.path.join(os.path.dirname(path_project), "output")

if sys.platform == "darwin":
    path_model = "/Users/lukasi33/project/LLM"
else:
    path_model = "F:/LLM"

## step-1: 载入 API KEY

In [4]:
load_dotenv(dotenv_path="explore.env")
deepseek_key = os.getenv("DEEPSEEK_KEY")
baidu_key = os.getenv("BAIDU_KEY")

## step-2: 实例化客户端

In [32]:
# 调用模型 API
# client = OpenAI(
#     api_key=deepseek_key,
#     base_url="https://api.deepseek.com/"
#     )

In [5]:
# 本地 Ollama
client = OpenAI(
    base_url="http://127.0.0.1:11434/v1",
    api_key="EMPTY",
    timeout=60
    )

In [6]:
swarm_client = Swarm(client)

## step-3: Agent 推理

In [7]:
# checkpoint = "deepseek-chat"  # v3
# checkpoint = "deepseek-reasoner"  # r1
checkpoint = "qwen2.5:7b"

In [14]:
agent = Agent(
    name="Master Agent",
    model=checkpoint,
    instructions="You are a helpful assistant on business travel."
)

In [15]:
system_prompt = (
    "你叫小慧助手，是由Lukas开发的差旅智能客服。"
    "你的身份是一名差旅秘书，"
    "你的任务是为用户提供基础对话、差旅知识问答、酒店推荐服务。"
    "当问及你的模型参数时，标准回答是属于公司保密信息，要强调模型设计的高效，能够提供高质量的服务。"
    "You are a helpful assistant on business travel."
)

In [16]:
user_prompt = "你好呀~"

In [17]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [18]:
response = swarm_client.run(
    agent=agent,
    messages=messages
)

In [19]:
print(response.messages[-1]["content"])

您好！很高兴为您服务。请问在差旅方面有什么可以帮助您的吗？无论是预订酒店、了解差旅知识还是解决问题，我都将尽力为您提供帮助。


## step-4: 工具调用

In [20]:
cityName2districtId = {
    "南京": "320100",
    "深圳": "440300"
}

In [21]:
def get_weather(cityName):
    districtId = cityName2districtId.get(cityName)
    url = f"https://api.map.baidu.com/weather/v1/?district_id={districtId}&data_type=all&ak={baidu_key}"
    response = requests.get(url)
    data = response.json()
    return json.dumps(data)

In [22]:
# test tool
data = get_weather(cityName="南京")
data

'{"status": 0, "result": {"location": {"country": "\\u4e2d\\u56fd", "province": "\\u6c5f\\u82cf\\u7701", "city": "\\u5357\\u4eac\\u5e02", "name": "\\u5357\\u4eac", "id": "320100"}, "now": {"text": "\\u6674", "temp": 9, "feels_like": 8, "rh": 43, "wind_class": "2\\u7ea7", "wind_dir": "\\u4e1c\\u98ce", "uptime": "20250130203500"}, "forecasts": [{"text_day": "\\u591a\\u4e91", "text_night": "\\u9634", "high": 15, "low": 6, "wc_day": "3~4\\u7ea7", "wd_day": "\\u4e1c\\u5357\\u98ce", "wc_night": "3~4\\u7ea7", "wd_night": "\\u4e1c\\u98ce", "date": "2025-01-30", "week": "\\u661f\\u671f\\u56db"}, {"text_day": "\\u4e2d\\u96e8", "text_night": "\\u4e2d\\u96e8", "high": 9, "low": 7, "wc_day": "3~4\\u7ea7", "wd_day": "\\u4e1c\\u5317\\u98ce", "wc_night": "3~4\\u7ea7", "wd_night": "\\u5317\\u98ce", "date": "2025-01-31", "week": "\\u661f\\u671f\\u4e94"}, {"text_day": "\\u5c0f\\u96e8", "text_night": "\\u5c0f\\u96e8", "high": 9, "low": 4, "wc_day": "4~5\\u7ea7", "wd_day": "\\u5317\\u98ce", "wc_night": "3~

In [23]:
agent = Agent(
    name="Master Agent",
    model=checkpoint,
    instructions="You are a helpful assistant on business travel.",
    functions=[get_weather]
)

In [24]:
user_prompt = "帮我查下南京的天气"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [29]:
response = swarm_client.run(
    agent=agent,
    messages=messages
)

In [30]:
print(response.messages[-1]["content"])

南京今天的天气情况如下：
- 天气：多云，当前温度9°C，体感温度8°C。
- 湿度为43%，风力为2级，方向来自东北。

未来几天的天气预测：
- 1月30日（周二）白天多云晚上晴朗，最高气温15°C，最低气温6°C。
- 1月31日（周三）白天和夜间均为中等云量，最高气温9°C，最低气温7°C。
- 2月1日（周四）白天小云量晚上也小云量，最高气温9°C，最低气温4°C。
- 2月2日（周五）白天晴朗晚上多云，最高气温9°C，最低气温3°C。
- 2月3日（周六）白天和夜间都是多云，最高气温8°C，最低气温为-1°C。

请注意保暖哦！


## step-5: Multi-Agent